# Dolphin basic walkthrough

This notebook demonstrates the basic usage of the `dolphin` command line tool to execute the stack-based phase linking workflow.
In this notebook, we will

- Download geocoded, co-registered single-look complex (CSLC) radar images from [ASF](https://search.asf.alaska.edu/)
- Prepare a configuration file for a stack of coregistered single-look complex (SLC) radar images with `dolphin config`
- Run this configuration file with `dolphin run` 
- Inspect the resulting output interferograms
- Show the most common parameters you may want to change or customize



## Setup

We first need to install `dolphin` as outlined in the [Getting Started](https://dolphin-insar.readthedocs.io/en/getting-started) section of the documentation. 

If you are running this in Colab, you can install [`dolphin` using `pip`](https://pypi.org/project/dolphin/)


In [ ]:
!pip install dolphin

We can check that we have the command line tool correctly installed by running:

In [3]:
!dolphin --help

usage: dolphin [-h] [--version] {run,config,unwrap} ...

options:
  -h, --help           show this help message and exit
  --version            show program's version number and exit

subcommands:
  {run,config,unwrap}


or by importing dolphin in python

In [1]:
import dolphin

dolphin.show_versions()

dolphin version: 0.26.0

Python deps:
        h5py: 3.11.0
         jax: 0.4.26
       numba: 0.58.1
       numpy: 1.26.2
 opera-utils: 0.6.0.post1.dev4+g7626988
    pydantic: 2.8.2
      pyproj: 3.6.1
    rasterio: 1.3.10
 ruamel.yaml: 0.17.21
       scipy: 1.13.0
threadpoolctl: 3.2.0
        tqdm: 4.66.1
  osgeo.gdal: 3.8.5

System:
      python: 3.11.6 | packaged by conda-forge | (main, Oct  3 2023, 10:37:07) [Clang 15.0.7 ]
  executable: /Users/staniewi/miniconda3/envs/mapping-311/bin/python
     machine: macOS-14.6.1-arm64-arm-64bit
optional GPU info:
         jax: 0.4.26
gpu_is_available: False


If you have a GPU available to you, you can follow the [extra installation set up](https://dolphin-insar.readthedocs.io/en/latest/gpu-setup/) so that the GPU verion of the workflow run.
This can be 5-20x faster than the CPU version, depending on the size of your workstation.
Here we will be processing a relatively small area, so the CPU will suffice.

## Input dataset

To find input data, you can use the [ASF search UI](https://search.asf.alaska.edu/) to explore and get a list of URLs to download; for our purposes, we will use the [OPERA Co-regisred Single-look Complex product](https://www.jpl.nasa.gov/go/opera/products/cslc-product-suite), which `dolphin` can directly process.

The helper functions in the [`opera-utils`](https://github.com/opera-adt/opera-utils) library provide wrappers over the [ASF library](https://github.com/asfadmin/Discovery-asf_search) to make it easy to download OPERA CSLCs over a certain region.

We will use a stack of Sentinel-1 SLCs from descending track 78 over West Texas, where wastewater injection lead to a huge jump in surface displacement in 2022:

In [ ]:
!pip install opera-utils asf_search

Since ASF requires a login to download data, you must add your username/password here:

In [ ]:
asf_username = "USERNAME-CHANGEME"
asf_password = "PASSWORD-CHANGEME"

In [ ]:
import subprocess
from contextlib import chdir
from pathlib import Path

import opera_utils.download

aoi = "POLYGON((-102.8136 31.3039,-102.5927 31.3039,-102.5927 31.532,-102.8136 31.532,-102.8136 31.3039))"
results, options = opera_utils.download.search_cslcs(
    aoi_polygon=aoi,
    # We want to have the same set of dates for each Burst ID (spatial footprint)
    check_missing_data=True,
    track=78,
    start="2021-06-01",
    end="2022-06-01",
)
best_option = options[0]

slc_dir = Path("input_slcs")
slc_dir.mkdir(exist_ok=True)
url_file = slc_dir / "urls.txt"
with open(url_file, "w") as f:
    f.write("\n".join(best_option.inputs))


with chdir(slc_dir):
    # Download 4 in parallel
    subprocess.run(f"cat urls.txt | xargs  -P4 -n1 wget --user {asf_username} --password {asf_password}", shell=True, check=True)

In the `input_slcs` directory, we have stored the NetCDF-format SLCs:

In [7]:
!ls input_slcs

t042_088905_iw1_20221107.h5  t042_088906_iw1_20221107.h5
t042_088905_iw1_20221119.h5  t042_088906_iw1_20221119.h5
t042_088905_iw1_20221201.h5  t042_088906_iw1_20221201.h5
t042_088905_iw1_20221213.h5  t042_088906_iw1_20221213.h5


See the [OPERA CSLC documentation](https://d2pn8kiwq2w21t.cloudfront.net/documents/OPERA_CSLC-S1_ProductSpec_v1.0.0_D-108278_Initial_2023-09-11_URS321269.pdf) for the full filename convention, but the main points are

- `T078` is Sentinel-1 track (relative orbit) 78
- 088905 the Burst IDs from [ESA's Burst database](https://sentinel.esa.int/web/sentinel/-/publication-of-brust-id-maps-for-copernicus-sentinel-1/1.1).
- `iw1` indicates these are from the first subswath (since the "Burst ID" is repeated for subswaths IW1,2,3.)
- `20221119` is the acquisition date formatted as `%Y%m%d`

Note that we specified the data we want is in `/data/VV`. This is not necessary for other SLC formats (e.g. binary files from ISCE2).

You can process one single stack, or multiple geocoded stacks. If you have different spatial regions, `dolphin` will form burst-wise interferograms and stitch them before unwrapping.

Let's make a configuration file for all of the bursts:

In [12]:
!dolphin config --slc-files input_slcs/*.h5 --subdataset "/data/VV"

Saving configuration to dolphin_config.yaml


If you need more fine-grained control of which SLCs to include, you can list the file locations in a text file separated by new lines and refer to it with an `@` symbol. For example: 

In [13]:
# Store the files we want in a text file called slc_list.txt
# Here we're just using `ls` to all the 185683 SLCs
!ls input_slcs/*h5 > slc_list.txt

# We use the same `--slc-files` argument, but now use an @ to say look inside the file
!dolphin config --slc-files @slc_list.txt -o new_config.yaml --subdataset "/data/VV"

Saving configuration to new_config.yaml


This is an equivalent way to point to the SLCs you want to process. The configs should be the same (except for the creation time, which is logged):

In [14]:
!diff new_config.yaml dolphin_config.yaml

162c162
< creation_time_utc: '2023-10-03T17:14:25.047623'
---
> creation_time_utc: '2023-10-03T17:14:17.143600'


This command created a YAML file in our current directory. Most of the contents were filled in by the workflow defaults:

In [16]:
!head -20 dolphin_config.yaml

input_options:
  # If passing HDF5/NetCDF files, subdataset to use from CSLC files. .
  #   Type: string | null.
  subdataset: /data/VV
  # Format of dates contained in CSLC filenames.
  #   Type: string.
  cslc_date_fmt: '%Y%m%d'
# REQUIRED: list of CSLC files, or newline-delimited file containing list of CSLC files.
#   Type: array.
cslc_file_list:
  - input_slcs/t042_088905_iw1_20221107.h5
  - input_slcs/t042_088906_iw1_20221107.h5
  - input_slcs/t042_088905_iw1_20221119.h5
  - input_slcs/t042_088906_iw1_20221119.h5
  - input_slcs/t042_088905_iw1_20221201.h5
  - input_slcs/t042_088906_iw1_20221201.h5
  - input_slcs/t042_088905_iw1_20221213.h5
  - input_slcs/t042_088906_iw1_20221213.h5
# Byte mask file used to ignore low correlation/bad data (e.g water mask). Convention is 0
#   for no data/invalid, and 1 for good data. Dtype must be uint8.


You can browse the YAML file for all the configuration options.

### Common configuration

#### Strides
You can create a downsampled version of the output using `--strides`. 
This will save time/space by creation an output with coarser pixel spacing than your SLCs.
For COMPASS outputs, adding `--strides 6 3` will convert the inputs at (5m, 10m) in `(x, y)` (aka `(easting, northing)`) to a (30 meter, 30 meter) output.


#### Specify the working directory

Use `--working-directory` to save all rasters to a different directory other than the one you call `dolphin run` from.

#### Specify how many CPUs to use

Use `--threads-per-worker` to specify the number of CPUs you would like the workflow to use.

####  (For geocoded SLCs) process different burst stacks in parallel

By adding the `--n-parallel-bursts`, you can process separate geocoded bursts at the same time (assuming sufficient resources are available).

#### Phase unwrapping

`dolphin` supports multiple options for phase unwrapping. Here, we will use the [Python wrapper for SNAPHU](https://github.com/isce-framework/snaphu-py), one of the most widely used phase unwrapping algorithms.


In [2]:
!pip install snaphu

/Users/staniewi/miniconda3/envs/mapping-311/lib/python3.11/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()



[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


### Full configuration command

In [21]:
cmd = (
    'dolphin config --slc-files input_slcs/*  --subdataset "/data/VV"  --strides 6 3 '
    "--n-parallel-bursts 2 --threads-per-worker 8 "
)
subprocess.run(cmd, shell=True, check=True)

Saving configuration to dolphin_config.yaml


CompletedProcess(args='dolphin config --slc-files @slc_list.txt  --subdataset "/data/VV"  --strides 6 3 --n-parallel-bursts 2 --threads-per-worker 16  --ntiles 2 2 --downsample-factor 3 3', returncode=0)

## Running the workflow

Now that we have created the `dolphin_config.yaml` file, we can run it using `dolphin run`

In [22]:
%%time
!dolphin run dolphin_config.yaml

[2023-10-03 10:33:47] INFO     Found SLC files from 2 bursts       ]8;id=381548;file:///u/aurora-r0/staniewi/repos/dolphin/src/dolphin/workflows/s1_disp.py\s1_disp.py]8;;\:]8;id=436871;file:///u/aurora-r0/staniewi/repos/dolphin/src/dolphin/workflows/s1_disp.py#68\68]8;;\
                      INFO     Running wrapped phase         ]8;id=597451;file:///u/aurora-r0/staniewi/repos/dolphin/src/dolphin/workflows/wrapped_phase.py\wrapped_phase.py]8;;\:]8;id=995919;file:///u/aurora-r0/staniewi/repos/dolphin/src/dolphin/workflows/wrapped_phase.py#39\39]8;;\
                               estimation in                                    
                               /u/aurora-r0/staniewi/dev/bet                    
                               a-delivery/delivery_data_smal                    
                               l/t042_088906_iw1                                
                      INFO     Running wrapped phase         ]8;id=227730;file:///u/aurora-r0/staniewi/

### Outputs

For each stack of SLCs (which may be > 1 when processing COMPASS GSLCs), the workflow creates a folder for
1. persistent scatter outputs (`PS`)
2. linked phase optimized SLCs (`linked_phase`)
3. (virtual) interferograms formed using the optimized SLCs (`interferograms`)

Here we have two of these subdirectories named `t042_088905_iw1` and `t042_088906_iw1`.
Additionally, you may notice
- The `slc_stack.vrt` is a VRT file pointing to the input SLCs for that burst stack.
- The `nodata_mask.tif` has been created from the COMPASS GSLC metadata to skip over the nan regions

Last, there is a top-level directory for `interferograms` that have been stitched together, and an `unwrapped` folder for the outputs of phase unwrapping.

```
$ tree -L 2
.
├── dolphin_config.yaml
├── input_slcs
│   ├── t042_088905_iw1_20221107.h5
│   ├── t042_088905_iw1_20221119.h5
│   ├── t042_088905_iw1_20221201.h5
│   ├── t042_088905_iw1_20221213.h5
│   ├── t042_088906_iw1_20221107.h5
│   ├── t042_088906_iw1_20221119.h5
│   ├── t042_088906_iw1_20221201.h5
│   └── t042_088906_iw1_20221213.h5
├── interferograms
│   └── stitched
├── new_config.yaml
├── slc_list.txt
├── t042_088905_iw1
│   ├── interferograms
│   ├── linked_phase
│   ├── nodata_mask.tif
│   ├── PS
│   ├── slc_stack.vrt
│   └── unwrapped
├── t042_088906_iw1
│   ├── interferograms
│   ├── linked_phase
│   ├── nodata_mask.tif
│   ├── PS
│   ├── slc_stack.vrt
│   └── unwrapped
└── unwrapped
    ├── 20221107_20221119.unw.conncomp
    └── 20221107_20221119.unw.tif
    └── ...

```

## Visualization the displacement

The outputs can be plotted using any tool capable of reading GDAL-compatible rasters. You can also use the `dolphin.io.load_gdal` function for convenience.


In [49]:
file_list = sorted(Path("timeseries/").glob("*.unw.tif"))
print(f"Found {len(file_list)} timeseries files")

velocity_file = next(Path("timeseries/").glob("velocity*tif"))

Found 3 interferograms
Found 3 correlation files
Found 3 unwrapped interferograms


In [ ]:
from dolphin.io import load_gdal
import matplotlib.pyplot as plt

%matplotlib inline

velocity = load_gdal(velocity_file)

fig, ax = plt.subplots()
ax.imshow(velocity)

